# Vẽ UML


- Xác định các entities
- Xác định attibute của các entities
- Xác đĩnh các relations giữa các entities

http://staruml.io/

# Vẽ wireframe

https://www.draw.io/

# Models

In [ ]:
# Models: Board, Topic, Post
# User model is already defined inside a builtin-app named auth
from django.db import models
from django.contrib.auth.models import User


class Board(models.Model):
    name = models.CharField(max_length=30, unique=True)
    description = models.CharField(max_length=100)


class Topic(models.Model):
    subject = models.CharField(max_length=255)
    last_updated = models.DateTimeField(auto_now_add=True)
    board = models.ForeignKey(Board, related_name='topics')
    starter = models.ForeignKey(User, related_name='topics')


class Post(models.Model):
    message = models.TextField(max_length=4000)
    topic = models.ForeignKey(Topic, related_name='posts')
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(null=True)
    created_by = models.ForeignKey(User, related_name='posts')
    updated_by = models.ForeignKey(User, null=True, related_name='+')

Django tự động tạo reverese relationships

`related_name`: là một optional. Nếu ta không set name cho nó, Django sẽ tự động tạo reverse name: `(class_name)_set`

Ví dụ trong `Board` model, thì các `Topic` sẽ được gọi dưới dạng `topic_set` property

Thay vì vậy, ta renamed nó lại thành `topics` để làm tự nhiên hơn

Trong `Post` model, `updated_by` field sets bằng `related_name = '+'`. Điều này dẫn đến Django không cần reverse relationships, nó sẽ ignore đi

# Experimenting with the Models API

In [ ]:
# interactive shell
python manage.py shell


In [ ]:
# Create object by using save()
from boards.models import Board
board = Board(name='Django', description='This is a board about Django.')
board.save()

Save method sẽ giúp ta create và update lại các objects.

In [ ]:
# Create directly object
board = Board.objects.create(name='Python', description='General discussion about Python.')


# Summary of Models Operations

Create an object without saving

`board = Board()`

Save an object (create or update)

`board.save()`

Create and save an object in the database

`Board.objects.create(name='...', 
description='...')`

List all objects

`Board.objects.all()`

Get a single object, identified by a field

`Board.objects.get(id=1)`

# Testing the Homepage

In [ ]:
from django.core.urlresolvers import reverse
from django.test import TestCase

class HomeTests(TestCase):
    def test_home_view_status_code(self):
        url = reverse('home')
        response = self.client.get(url)
        self.assertEquals(response.status_code, 200)

- Simple Test case nhưng useful

- Test trên dùng kiểm tra status code của response

- Status code 200 nghĩa là `success`

- Status code nếu hiện 500 tất là `Internal Server Error`

In [ ]:
from django.core.urlresolvers import reverse
from django.urls import resolve
from django.test import TestCase
from .views import home

class HomeTests(TestCase):
    def test_home_view_status_code(self):
        url = reverse('home')
        response = self.client.get(url)
        self.assertEquals(response.status_code, 200)

    def test_home_url_resolves_home_view(self):
        view = resolve('/')
        self.assertEquals(view.func, home)

Trong test thứ hai, ta thêm vào `resolve` function

- Django dùng nó để match một URL được requested nằm trong list các URLs listed trong `urls.py`

- Test này đảm bảo URL `/` sẽ trả về `views.home`

In [ ]:
# Running test
python manage.py test
# Running test if you want to see verbose
python manage.py test --verbosity=2

Verbosity determines the amount of notification and debug information that will be printed to the console; 0 is no output, 1 is normal output, and 2 is verbose output.

# Using static files (javascript, css, jQuery, ...)

myproject/
 |-- myproject/
 |    |-- boards/
 |    |-- myproject/
 |    |-- templates/
 |    |-- static/
 |    |    +-- css/
 |    |         +-- bootstrap.min.css    <-- here
 |    +-- manage.py
 +-- venv/


Ta thêm vào statics.py như sau:

In [ ]:
STATIC_URL = '/static/'

STATICFILES_DIRS = [
    os.path.join(BASE_DIR, 'static'),
]

Same thing as the TEMPLATES directory, remember

In [ ]:
# templates/home.html
{% load static %}<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
    <title>Boards</title>
    <link rel="stylesheet" href="{% static 'css/bootstrap.min.css' %}">
  </head>
  <body>
    <!-- body suppressed for brevity ... -->
  </body>
</html>

Ghi nhớ: Ta luôn để `{% load static %}` lên đầu template

Template tag `{% static %}` được sử dụng bao gồm URL nơi mà chứa các resource lives

Trong quá trình debug, `{% static 'css/bootstrap.min.css' %}` sẽ trả về __/static/css/bootstrap.min.css__ tương đương __http://127.0.0.1:8000/static/css/bootstrap.min.css__

Trong thực tế, deploy trang web lên server, cấu hình trong settings.py bao gồm URL final.

Ví dụ, hosted của các static files của bạn là __https://static.examples.com/__ thì `{% static 'css/bootstrap.min.css' %}` trả về __https://static.example.com/css/bootstrap.min.css__

# Using Django Admin

Let’s start by creating an administrator account:

In [ ]:
python3 manage.py createsuperuser

Follow the instructions:

In [ ]:
Username (leave blank to use 'vitorfs'): admin
Email address: admin@example.com
Password:
Password (again):
Superuser created successfully.

Muốn quản lý các model, sử dụng `django.contrib.admin`

In [ ]:
# boards/admin.py
from django.contrib import admin
from .models import Board

admin.site.register(Board)